In [10]:
from langchain_docling import DoclingLoader
import re
from typing import List, Dict, Any
import os

def load_docling_chapters(path: str) -> List[Dict[str, str]]:
    """위키독스 텍스트 파일을 로드하고 챕터별로 분할"""
    # 파일을 직접 읽기
    try:
        with open(path, 'r', encoding='cp949') as f:
            document = f.read()
    except UnicodeDecodeError:
        # cp949로 안되면 euc-kr 시도
        try : 
            with open(path, 'r', encoding='euc-kr') as f:
                document = f.read()
        except UnicodeDecodeError:
            with open(path, 'r', encoding='utf-8') as f:
                document = f.read()            
    
    # 챕터 정보 추출
    chapters = []
    current_chapter = None
    current_content = []
    
    # 문서의 각 줄을 순회
    for line in document.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        # 챕터 제목 확인 (숫자. 제목 형식)
        if re.match(r'^\d+\.\s+', line):
            # 이전 챕터가 있다면 저장
            if current_chapter:
                chapters.append({
                    'title': current_chapter,
                    'content': '\n'.join(current_content)
                })
            
            # 새 챕터 시작
            current_chapter = line
            current_content = []
        else:
            # 현재 챕터의 내용 추가
            current_content.append(line)
    
    # 마지막 챕터 저장
    if current_chapter:
        chapters.append({
            'title': current_chapter,
            'content': '\n'.join(current_content)
        })
    
    return chapters

In [12]:
# 파일 경로 설정
file_path = "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_book_2155.txt"

# 챕터 로드
first_docs = load_docling_chapters(file_path)

# # 결과 출력
# print(f"총 {len(first_docs)}개의 챕터가 발견되었습니다.")
# for i, chapter in enumerate(first_docs, 1):
#     print(f"\n챕터 {i}:")
#     print(f"제목: {chapter['title']}")
#     print(f"내용 길이: {len(chapter['content'])}")
#     print("-" * 50)

# # 챕터 내용 확인
# print("\n첫 번째 챕터 내용 미리보기:")
# if first_docs:
#     first_chapter = first_docs[0]
#     print(f"제목: {first_chapter['title']}")
#     print("내용:")
#     print(first_chapter['content'][:500] + "...")  # 처음 500자만 출력

In [13]:
# 파일 경로 설정
file_path = "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_book_2788.txt"

# 챕터 로드
second_docs = load_docling_chapters(file_path)

In [ ]:
# 파일 경로 설정
file_path = "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_book_14314.txt"

# 챕터 로드
third_docs = load_docling_chapters(file_path)

In [21]:
print(len(first_docs))
print(len(second_docs))
print(len(third_docs))

538
417
776


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크 크기 설정
    chunk_overlap=100,  # 청크 중복 설정
    separators=["\n\n", "\n", ". ", " ", ""]
)

for doc in first_docs :
    splits = splitter.split_text(doc['content'])
    print(splits)




['링크 : https://www.anaconda.com/distribution/\n[이미지: ]\n위 사이트 링크로 이동하여 사이트 하단으로 이동하면 (저자가 이 책을 작성할 당시 기준) 좌측에 파이썬 3.7 버전과 우측에 파이썬 2.7 버전의 아나콘다 설치 파일이 있습니다. 여기서는 파이썬 3.7 버전 64 비트를 설치합니다.\n[이미지: ]\n설치 파일을 실행한 후에 다른 윈도우 프로그램을 설치하는 것과 마찬가지로 Next > 를 누르면서 설치를 완료하면 됩니다. 아나콘다를 설치하면 머신 러닝을 위한 기본적인 파이썬 패키지들은 자동으로 설치가 됩니다. 다만 텐서플로우, 케라스, 젠심, 코엔엘파이와 같은 패키지들은 별도 설치가 필요한데 이에 대해서는 뒤에서 추가적으로 설명하겠습니다.\n아나콘다를 다 설치했다면 아나콘다 프롬프트를 오픈하겠습니다.\n[이미지: ]\n아나콘다 프롬프트를 열었다면 아나콘다 프롬프트에 아래의 커맨드를 입력하여 아나콘다 파이썬 패키지를 전부 최신 버전으로 업데이트 합니다.', '[이미지: ]\n아나콘다 프롬프트를 열었다면 아나콘다 프롬프트에 아래의 커맨드를 입력하여 아나콘다 파이썬 패키지를 전부 최신 버전으로 업데이트 합니다.\n> conda update -n base conda\n> conda update --all\n이 책이 작성되었을 당시에는 파이썬 3.7 버전이 최신 버전이었지만, 독자분들이 파이썬을 설치하기 위해서 아나콘다 페이지에 접속하였을 때는 3.7보다 더욱 최신 버전으로 업데이트가 되었을 수 있습니다. 이 경우 무작정 파이썬 최신 버전을 설치하는 것은 좋은 방법이 아닙니다. 일반적으로 아래의 링크에서 파이썬 버전과 호환되는 텐서플로우 버전에 대한 안내가 나와있으니 반드시 설치 전 확인이 필요합니다.\n링크 : https://www.tensorflow.org/install/pip?hl=ko', "링크 : https://www.tensorflow.org/install/pip?hl=ko\n예를 들어 위 페이지에서 'Pyth